In [47]:
import sys, os, importlib.util, pandas
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import lambda2nu, nu2lambda, c

In [48]:

def adguide(nf, nc, ns, lmbd, a, mode):
  mmax = 1000
  tol = 1e-10
  r = 0.3

  if (mode=='TM'):
    pc = (nf/nc)**2
    ps = (nf/ns)**2
  else:
    pc = 1
    ps = 1

  delta = (ns**2-nc**2)/(nf**2-ns**2)
  k0 = 2*np.pi/lmbd
  NA = np.sqrt(nf**2-ns**2)
  R = k0*a*NA

  M = int(np.ceil((2*R-np.arctan(pc*np.sqrt(delta)))/np.pi))
  m = np.arange(0,M)

  u = R*np.ones(M)
  v = np.zeros(M)
  w = R*np.sqrt(delta)*np.ones(M)

  Nit = 1

  while True:
    F = 0.5*(np.pi*m + np.arctan(ps*v/u) + np.arctan(pc*w/u))
    u_new = r*F + (1-r)*u
    if np.any(np.abs(u_new-u) <= tol):
        break
    Nit = Nit + 1
    u = u_new
    v = np.sqrt(R**2-u**2)
    w = np.sqrt(delta*R**2+v**2)
    if Nit > mmax:
        break

  kf = u/a
  gamma_s = v/a
  gamma_c = w/a
  be = np.sqrt((nf*k0)**2 - kf**2)
  return be,be/k0, kf, gamma_s, gamma_c, M
    

In [49]:
from IPython.core.display import HTML
from IPython.core.pylabtools import figsize
from IPython.display import display, Math

In [50]:
um=10e-6
nm=10e-9

In [51]:
lambda_0 = 1550*nm
n_c = 1.444 #@ 1550nm
n_r = 3.476 #@ 1550nm
k_0 = 2*np.pi/lambda_0
W_m = 7*um

In [52]:
sigma = 0 # TE mode
We = W_m + (lambda_0/np.pi)*( (n_c/n_r)**(2*0))*(n_r**2 - n_c**2)**(-0.5)
We/um

7.156040531990452

In [53]:
neff_app = np.zeros(10)
for i in range(10):
    neff_app[i] = n_r - ((i+1)**2*np.pi*lambda_0)/(We**2*4*n_r*k_0)
    k_C = ((i+1)*np.pi)/We
    beta=neff_app*k_0
    print("neff_app = %.4f"%(neff_app[i]))
    print("beta = %.4f"%(beta[i]))
    


neff_app = 3.4743
beta = 1408371.0703
neff_app = 3.4693
beta = 1406319.3532
neff_app = 3.4608
beta = 1402899.8246
neff_app = 3.4490
beta = 1398112.4847
neff_app = 3.4338
beta = 1391957.3333
neff_app = 3.4153
beta = 1384434.3705
neff_app = 3.3933
beta = 1375543.5963
neff_app = 3.3680
beta = 1365285.0107
neff_app = 3.3393
beta = 1353658.6137
neff_app = 3.3073
beta = 1340664.4052


In [54]:
Lpi_app = np.pi/(beta[0] - beta[1])
Lpi_app/um

np.float64(153.12016546058257)

In [55]:
Lpi_app = np.pi/(neff_app[0]-neff_app[1])/k_0
Lpi_app/um

np.float64(153.12016546058524)

In [56]:
l_pi_app = (4*n_r*We**2)/(3*lambda_0)
print(l_pi_app/um)
(3/2)*(l_pi_app/um)

153.12016546057967


229.68024819086952

    Returns:
        dict: Dictionary containing the computed parameters:
            - kf: Transverse wavenumbers inside slab in units of k₀
            - as_: Decay wavenumber in substrate in units of k₀
            - ac: Decay wavenumber in cladding in units of k₀
            - be: Beta (propagation wavenumbers in units of k₀)
            - Rm: Cutoff radius for m-th mode
            - fm: Cutoff frequencies in units of f = c₀/λ₀
            - thm: Angles of total internal reflection
            - Nit: Number of iterations it took to converge

        'be': be,
        'ne' :ne,
        'kf': kf,
        'as_': as_,
        'ac': ac,
        'Rm': Rm,
        
    }
    """

In [57]:
betaTE, neffTE, kFTE, alsTE, alcTE, MTE = adguide(n_r, n_c, n_c, lambda_0, W_m/2,'TE') 
print('tamanho =',len(betaTE))
print(neffTE)

tamanho = 29
[3.47431248 3.46924515 3.46078371 3.44890405 3.43357197 3.41474262
 3.39235988 3.36635548 3.33664787 3.3031409  3.2657221  3.22426059
 3.17860448 3.12857766 3.07397572 3.01456088 2.95005551 2.88013377
 2.80441071 2.72242787 2.63363394 2.5373582  2.43277357 2.31884399
 2.1942487  2.05727348 1.90566742 1.73655705 1.54760274]


In [58]:
df = pandas.DataFrame({"beta": betaTE,
                       "neff": neffTE,
                       "kF": kFTE,
                       "als": alsTE,
                       "alc": alcTE,})
df.index.names = ['TE']
df

,beta,neff,kF,als,alc
TE,,,,,
0,1.408371e+06,3.474312,4.390109e+04,1.280966e+06,1.280966e+06
1,1.406317e+06,3.469245,8.780104e+04,1.278707e+06,1.278707e+06
2,1.402887e+06,3.460784,1.316987e+05,1.274934e+06,1.274934e+06
3,1.398071e+06,3.448904,1.755930e+05,1.269633e+06,1.269633e+06
4,1.391856e+06,3.433572,2.194826e+05,1.262786e+06,1.262786e+06
5,1.384223e+06,3.414743,2.633664e+05,1.254368e+06,1.254368e+06
6,1.375150e+06,3.392360,3.072430e+05,1.244349e+06,1.244349e+06
7,1.364609e+06,3.366355,3.511110e+05,1.232689e+06,1.232689e+06
8,1.352566e+06,3.336648,3.949689e+05,1.219345e+06,1.219345e+06


In [59]:
L_pi_app = np.pi/(betaTE[0] - betaTE[1])
L_pi_app/um

np.float64(152.9407445649724)

In [60]:
L_pi_2x2 = ((3/(4*2))*L_pi_app)
L_pi_2x2/um

np.float64(57.35277921186466)

In [61]:
L_pi_2x1 = 3/8*L_pi_app
L_pi_2x1/um

np.float64(57.35277921186466)